In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glovetwitter27b100dtxt/glove.twitter.27B.100d.txt
/kaggle/input/twitterdata/finalSentimentdata2.csv


In [6]:
sentiment_df = pd.read_csv('/kaggle/input/twitterdata/finalSentimentdata2.csv')

In [7]:
from string import punctuation
from nltk.corpus import stopwords
print(stopwords.words('english')[10:15])

def punctuation_stopwords_removal(sms):
    # filters charecter-by-charecter : ['h', 'e', 'e', 'l', 'o', 'o', ' ', 'm', 'y', ' ', 'n', 'a', 'm', 'e', ' ', 'i', 's', ' ', 'p', 'u', 'r', 'v', 'a']
    remove_punctuation = [ch for ch in sms if ch not in punctuation]
    # convert them back to sentences and split into words
    remove_punctuation = "".join(remove_punctuation).split()
    filtered_sms = [word.lower() for word in remove_punctuation if word.lower() not in stopwords.words('english')]
    return filtered_sms

["you've", "you'll", "you'd", 'your', 'yours']


In [8]:
sentiment_df.loc[:, 'text'] = sentiment_df['text'].apply(punctuation_stopwords_removal)

In [9]:
reviews_split = []
for i, j in sentiment_df.iterrows():
    reviews_split.append(j['text'])

In [10]:
words = []
for review in reviews_split:
    for word in review:
        words.append(word)

In [11]:
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word:ii for ii, word in enumerate(vocab, 1)}

In [12]:
encoded_reviews = []
for review in reviews_split:
    encoded_reviews.append([vocab_to_int[word] for word in review])

In [13]:
labels_to_int = []
for i, j in sentiment_df.iterrows():
    if j['sentiment']=='joy':
        labels_to_int.append(1)
    else:
        labels_to_int.append(0)

In [14]:
reviews_len = Counter([len(x) for x in encoded_reviews])
print(max(reviews_len))

48


In [15]:
non_zero_idx = [ii for ii, review in enumerate(encoded_reviews) if len(encoded_reviews)!=0]
encoded_reviews = [encoded_reviews[ii] for ii in non_zero_idx]
encoded_labels = np.array([labels_to_int[ii] for ii in non_zero_idx])

In [16]:
def pad_features(reviews_int, seq_length):
    features = np.zeros((len(reviews_int), seq_length), dtype=int)
    for i, row in enumerate(reviews_int):
        if len(row)!=0:
            features[i, -len(row):] = np.array(row)[:seq_length]
    return features

In [17]:
seq_length = 50
padded_features= pad_features(encoded_reviews, seq_length)
print(padded_features[:2])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0  853  186   20 1079 1457 4429 2201  407 1240 1079
    15  218  337  167  253  462  337  122  168 4430 4431  140   23  264
    58  765    3    5  195 1079 2966  274]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0   80 1755 4432 2967
  4433   86  854 1080 2968 4434 4435    7]]


In [18]:
split_frac = 0.8
split_idx = int(len(padded_features)*split_frac)

training_x, remaining_x = padded_features[:split_idx], padded_features[split_idx:]
training_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [20]:
# torch.from_numpy creates a tensor data from n-d array
train_data = TensorDataset(torch.from_numpy(training_x), torch.from_numpy(training_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))

batch_size = 1

train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
valid_loader = DataLoader(valid_data, batch_size=batch_size)

In [21]:
gpu_available = torch.cuda.is_available

if gpu_available:
    print('Training on GPU')
else:
    print('GPU not available')

Training on GPU


In [28]:
import torch.nn as nn

class Bidirectional_LSTM_with_GloVeTwitter_Embeddings(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, embedding_matrix, drop_prob=0.2):
        super(Bidirectional_LSTM_with_GloVeTwitter_Embeddings, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_layer.weight = nn.Parameter(embedding_matrix, requires_grad=False)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, bidirectional=True, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
    
    def forward(self, x, hidden):
        # x : batch_size * seq_length * features
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding_layer(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sig(out)
        
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        # initialize weights for lstm layer
        weights = next(self.parameters()).data
        
        if gpu_available:
            hidden = (weights.new(self.n_layers*2, batch_size, self.hidden_dim).zero_().cuda(),
                     weights.new(self.n_layers*2, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weights.new(self.n_layers*2, batch_size, self.hidden_dim).zero_(),
                     weights.new(self.n_layers*2, batch_size, self.hidden_dim).zero())
        return hidden

In [29]:
#load fasttext embeddings
import codecs

print('loading word embeddings...')
glovetwitter_embedding = {}
f = codecs.open('/kaggle/input/glovetwitter27b100dtxt/glove.twitter.27B.100d.txt', encoding='utf-8')
for line in f:
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glovetwitter_embedding[word] = coefs
f.close()

loading word embeddings...


In [30]:
import tqdm

num_words=len(vocab_to_int)+1
glove_twitter_embedding_matrix = np.zeros((num_words,100))

for word,i in (vocab_to_int.items()):
    if i > num_words:
        continue
    
    emb_vec=glovetwitter_embedding.get(word)
    if emb_vec is not None:
        glove_twitter_embedding_matrix[i]=emb_vec

In [31]:
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1 # either happy or sad
embedding_dim = 100
hidden_dim = 256
n_layers = 2

bilstm_with_glove_twitter_embeddings = Bidirectional_LSTM_with_GloVeTwitter_Embeddings(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, torch.Tensor(glove_twitter_embedding_matrix))
bilstm_with_glove_twitter_embeddings.embedding_layer.weight.data = torch.Tensor(glove_twitter_embedding_matrix).cuda()

print(bilstm_with_glove_twitter_embeddings)

lr = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(bilstm_with_glove_twitter_embeddings.parameters(), lr=lr)

epochs = 4
count = 0
print_every = 100
clip = 5 
if gpu_available:
    bilstm_with_glove_twitter_embeddings.cuda()

bilstm_with_glove_twitter_embeddings.train()
for e in range(epochs):
    # initialize lstm's hidden layer 
    h = bilstm_with_glove_twitter_embeddings.init_hidden(batch_size)
    for inputs, labels in train_loader:
        count += 1
        if gpu_available:
            inputs, labels = inputs.cuda(), labels.cuda()
        h = tuple([each.data for each in h])
        
        # training process
        bilstm_with_glove_twitter_embeddings.zero_grad()
        outputs, h = bilstm_with_glove_twitter_embeddings(inputs, h)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm(bilstm_with_glove_twitter_embeddings.parameters(), clip)
        optimizer.step()
        
        # print average training losses
        if count % print_every == 0:
            val_h = bilstm_with_glove_twitter_embeddings.init_hidden(batch_size)
            val_losses = []
            bilstm_with_glove_twitter_embeddings.eval()
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                if gpu_available:
                    inputs, labels = inputs.cuda(), labels.cuda()
            outputs, val_h = bilstm_with_glove_twitter_embeddings(inputs, val_h)
            val_loss = criterion(outputs.squeeze(), labels.float())
            val_losses.append(val_loss.item())
        
            bilstm_with_glove_twitter_embeddings.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(count),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Bidirectional_LSTM_with_GloVeTwitter_Embeddings(
  (embedding_layer): Embedding(10663, 100)
  (lstm): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 1/4... Step: 100... Loss: 0.109153... Val Loss: 0.212390
Epoch: 1/4... Step: 200... Loss: 0.114740... Val Loss: 0.248327
Epoch: 1/4... Step: 300... Loss: 0.156756... Val Loss: 0.293008
Epoch: 1/4... Step: 400... Loss: 0.335014... Val Loss: 0.414086
Epoch: 1/4... Step: 500... Loss: 1.063716... Val Loss: 0.467379
Epoch: 1/4... Step: 600... Loss: 0.350736... Val Loss: 0.450590
Epoch: 1/4... Step: 700... Loss: 1.694402... Val Loss: 0.556861
Epoch: 1/4... Step: 800... Loss: 1.302303... Val Loss: 0.521525
Epoch: 1/4... Step: 900... Loss: 0.884050... Val Loss: 0.600392
Epoch: 1/4... Step: 1000... Loss: 0.047599... Val Loss: 0.491401
Epoch: 1/4... Step: 1100... Loss: 0.028393... Val Loss: 0.468005
Epoch: 1/4... Step: 1200... Loss: 2.365734... Val Loss: 0.400009
Epoch: 1/4... Step: 1300... Loss: 0.360712... Val Loss: 0.439259
Epoch: 1/4... Step: 1400... Loss: 0.265293... Val Loss: 0.492754
Epoch: 1/4... Step: 1500... Loss: 1.960674... Val Loss: 0.313198
Epoch: 1/4... Step: 1600... Loss: 

In [32]:
test_losses = []
num_correct = 0

h = bilstm_with_glove_twitter_embeddings.init_hidden(batch_size)
bilstm_with_glove_twitter_embeddings.eval()

for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    if gpu_available:
        inputs, labels = inputs.cuda(), labels.cuda()
    
    outputs, h = bilstm_with_glove_twitter_embeddings(inputs, h)
    test_loss = criterion(outputs.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(outputs.squeeze())
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not gpu_available else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

# printing average statistics
print("Test loss: {:.3f}".format(np.mean(test_losses)))
    
# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.264
Test accuracy: 0.922
